# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
keras = tf.keras
from sklearn.preprocessing import RobustScaler
import plotly.graph_objects as go
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Importing Data

In [ ]:
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

# Data Pre-Procesing

In [ ]:
recovered_df = recovered_df.groupby("Country/Region").sum().reset_index()
Country_recovered =  recovered_df['Country/Region'].values
recovered_df = recovered_df.drop(['Lat','Long','Country/Region'], axis=1)
recovered_df = recovered_df[recovered_df.columns].T
recovered_df.columns = Country_recovered
Date_index = recovered_df.index
recovered_df = recovered_df.diff().fillna(0)
recovered_df = recovered_df.abs()
recovered_df.head()

# Scalling the data

In [ ]:
sc = RobustScaler()
recovered_df_scaled = sc.fit_transform(recovered_df)
recovered_df_scaled = pd.DataFrame(recovered_df_scaled)

# Enter the number of days you want to predict

In [ ]:
DAYS_TO_PREDICT = int(input('Enter Number Of Days You Want to Predict : '))
predict_recovered_index = pd.date_range(start=Date_index[-1],periods=DAYS_TO_PREDICT + 1,closed='right')
predict_recovered_index = predict_recovered_index.strftime('%Y/%m/%d')

# Creating class for building model

In [ ]:
class Corona_recovered_model():
    def create_dataset(self, dataset, time_steps=1, column = 0):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_steps-1):
          a = dataset[i:(i+time_steps), column]
          dataX.append(a)
          dataY.append(dataset[i + time_steps, column])
        return np.array(dataX), np.array(dataY)

    def Model(self,dframe,column):
        df = dframe.values
        df = df.astype('float32')
        
        time_steps = 1
        trainX, trainY = self.create_dataset(df, time_steps, column)
        trainX = np.reshape(trainX, (trainX.shape[0],1,trainX.shape[1]))
        
        
        keras.backend.clear_session()
        tf.random.set_seed(42)
        np.random.seed(42)
        recover_model = keras.models.Sequential([
            keras.layers.InputLayer(input_shape=[None, 1]),
            keras.layers.Conv1D(filters=16, kernel_size=2,
                      strides=1, padding="causal",
                      activation="relu",
                      ),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(1,activation = 'linear',kernel_initializer = 'he_normal'),
            keras.layers.Lambda(lambda x: x * 200)
             ])
        
        optimizer = keras.optimizers.Adadelta(lr = 0.001)
        
        recover_model.compile(loss=['mae'],optimizer=optimizer)   
                
        recover_model.fit(trainX, trainY, epochs=100,
                          batch_size = 10, verbose = 0)

        recover_test_seq  = trainX[-1:]
        recover_test_seq = recover_test_seq
        recover_test_seq = recover_test_seq.reshape(len(recover_test_seq), time_steps, 1)
        recover_preds = []
        
        for _ in range (DAYS_TO_PREDICT):
            recover_pred = recover_model.predict(recover_test_seq)
            recover_preds.append(recover_pred)
            recover_new_seq = recover_pred
            recover_test_seq = recover_new_seq
            recover_test_seq = recover_test_seq.reshape(len(recover_test_seq ), time_steps, 1)

        

        return recover_preds

# Calling the class and predicting the cases for every individual country

In [ ]:
NN = Corona_recovered_model()
recovered_predictions = []
for i in range(len(Country_recovered)):
    result = NN.Model(recovered_df_scaled,i) 
    recovered_predictions.append(result)

In [ ]:
recovered_predictions = (np.array(recovered_predictions).transpose())
recovered_predictions = recovered_predictions.reshape(((DAYS_TO_PREDICT),len(Country_recovered)))
recovered_predictions = sc.inverse_transform(recovered_predictions)
recovered_predictions = pd.DataFrame(recovered_predictions)
recovered_predictions = recovered_predictions.abs()
recovered_predictions['Dates'] = predict_recovered_index
recovered_predictions.set_index('Dates', inplace = True)
recovered_predictions.columns = Country_recovered
recovered_predictions.head()

# Visualizing The Recovered Cases For A Country As Per The User's Choice

### List of Infected Countries

In [ ]:
Infected_Countries = pd.DataFrame(Country_recovered)
Infected_Countries.columns = ['Countries']
pd.set_option("max_rows", None)
Infected_Countries

### Select The Country

In [ ]:
Country = input('Enter the name of the country from above list you would like to see the daily cases till now : ')

## Till Date Daily Cases of Recovery from coronavirus

In [ ]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Recovered Daily Cases','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},
                  xaxis_title="Dates",
                  yaxis_title="Number Of Cases")
fig.add_trace(go.Scatter(x=recovered_df.index, 
                         y=recovered_df[Country],
                         mode='lines+markers',
                         line=dict(color='green', width=2)))

## Predictions of recovered cases from coronavirus

In [ ]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Recovered Cases Predictions','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},xaxis_title="Dates",yaxis_title="Number Of Cases")
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=recovered_predictions.index, 
                         y=recovered_predictions[Country],
                         mode='lines+markers',
                         line=dict(color='white', width=2)))